In [ ]:
# 📦 설치
!pip install transformers pandas openpyxl


In [ ]:
import pandas as pd

df = pd.read_excel("/content/kote.xlsx", engine="openpyxl")
print("✅ 컬럼명 목록:", df.columns.tolist())
df.head()



In [ ]:

# ✅ 라이브러리 불러오기
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
# ✅ 정확한 Colab 경로로 수정
xlsx_path = "/content/kote.xlsx"

# 파일 로딩
df = pd.read_excel(xlsx_path, engine="openpyxl")

# 컬럼명 확인
print("✔️ 컬럼명 목록:", df.columns.tolist())
df.head()


In [ ]:
# 🔎 실제 컬럼명에 따라 수정 필요
text_col = "사람문장1"  # 또는 '발화', '문장', 등

texts = df[text_col].dropna().astype(str).tolist()
texts = texts[:10]  # 예제용으로 10개만 처리 (더 하고 싶으면 이 줄 제거)

# ✅ 모델 로딩
model_name = "searle-j/kote_for_easygoing_people"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

pipe = TextClassificationPipeline(
    model=model,
    tokenizer=tokenizer,
    device=device,
    return_all_scores=True,
    function_to_apply='sigmoid'
)

# ✅ 감정 분석
pipe_outputs = pipe(texts)

# ✅ 출력 (score ≥ 0.4만)
for i in range(len(pipe_outputs)):
    print(f"\n▶ 문장: {texts[i]}")
    for output in pipe_outputs[i]:
        if output["score"] > 0.4:
            print(f"  - 감정: {output['label']} / 확률: {output['score']:.2f}")


In [ ]:
# ✅ 사용자 입력 문장 감정 분석
while True:
    user_input = input("감정 분석할 문장을 입력하세요 (종료하려면 exit): ")
    if user_input.lower() == "exit":
        break
    result = pipe(user_input)[0]
    print(f"▶ 입력 문장: {user_input}")
    for r in result:
        if r["score"] > 0.4:
            print(f"  - 감정: {r['label']} / 확률: {r['score']:.2f}")
    print()
